# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [4]:
from xml.etree import ElementTree as ET
import pandas as pd
import numpy as np

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [5]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [6]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [7]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print ('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print (capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [8]:
document = ET.parse( './data/mondial_database.xml' )

In [9]:
infmort = []
countryname = []

for child in document.getroot():
    if (child.find('infant_mortality') != None) & (child.find('name') != None) :
       
        infmort.append(child.find('infant_mortality').text)
        countryname.append(child.find('name').text)       
df = pd.DataFrame({'country' : countryname,
     'mortality' : infmort})
df['mortality'] = df['mortality'].astype(float)
df = df.sort('mortality', ascending = True)
df.head(10)

C:\Users\Nolo42\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,country,mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


In [10]:
droot=document.getroot()

In [11]:
pop = []
city = []
for element in document.iterfind('country'):
    for subelement in element.iter('city'):
        city.append(subelement.find('name').text)
        if(subelement.find('population') != None):
            pop.append(subelement.findall('population')[-1].text)
        else:
            pop.append(np.nan)            
            

df = pd.DataFrame({'city' : city, 'population' : pop})
df['population'] = df['population'].astype(float)
df = df.sort('population', ascending = False)
df.head(10)

C:\Users\Nolo42\Anaconda3\lib\site-packages\ipykernel\__main__.py:14: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,city,population
1341,Shanghai,22315474
771,Istanbul,13710512
1527,Mumbai,12442373
479,Moskva,11979529
1340,Beijing,11716620
2810,São Paulo,11152344
1342,Tianjin,11090314
1064,Guangzhou,11071424
1582,Delhi,11034555
1067,Shenzhen,10358381


In [12]:
countrypop= []
ethnicity = []
percente = []

for child in document.getroot():
    if (child.find('ethnicgroup') != None):
        for i in range(len(child.findall('ethnicgroup'))):
            countrypop.append(child.findall('population')[-1].text)
            ethnicity.append(child.findall('ethnicgroup')[i].text)
            (percente).append(child.findall('ethnicgroup')[i].attrib['percentage'])

df = pd.DataFrame({'ethnic group' : ethnicity, 'percentage' : percente, 'population' : countrypop })
df['population'] = df['population'].astype(float)
df['percentage'] = df['percentage'].astype(float)
df['ethnic count'] = df['population'] * df['percentage'] /100
df.groupby('ethnic group')['ethnic count'].sum().order('ethnic count', ascending = False).head(10)

C:\Users\Nolo42\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: FutureWarning: order is deprecated, use sort_values(...)


ethnic group
Han Chinese    1.245059e+09
Indo-Aryan     8.718156e+08
European       4.948722e+08
African        3.183251e+08
Dravidian      3.027137e+08
Mestizo        1.577344e+08
Bengali        1.467769e+08
Russian        1.318570e+08
Japanese       1.265342e+08
Malay          1.219936e+08
Name: ethnic count, dtype: float64